In [1]:
 import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [2]:
 from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("amazon").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [3]:
 # Read in data from S3 Buckets
from pyspark import SparkFiles
# loading data from S3 
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)



In [4]:
#read csv with spark
electronics_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", timestampFormat="yyyy/mm/dd", inferSchema=True)

# Show DataFrame
electronics_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [5]:
electronics_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [6]:
 # Drop null values
print(electronics_df.count())
dropna_df = electronics_df.dropna()

print(dropna_df.count())

3093869
3093660


In [7]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
cleaned_df = dropna_df.filter(col("verified_purchase")  == 'Y')
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [8]:
#verified purchase only
print(cleaned_df.count())

2600049


In [9]:
#Create dataframes for import into PostgreSQL
review_df = cleaned_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2MTG1GCZLR2DK|   41409413|B00428R89M|     112201306| 2015-08-31|
|R2HBOEM8LE9928|   49668221|B000068O48|     734576678| 2015-08-31|
|R1P4RW1R9FDPEE|   12338275|B000GGKOG8|     614448099| 2015-08-31|
|R1EBPM82ENI67M|   38487968|B000NU4OTA|      72265257| 2015-08-31|
|R372S58V6D11AT|   23732619|B00JOQIO6S|     308169188| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [10]:
products_df = cleaned_df.select(['product_id','product_title']).distinct()
products_df.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00JL2498C|Jensen JTA-410-BL...|
|B00OUKG3NE|Sennheiser Androi...|
|B0083H9R6A|Electrified ELPLP...|
|B0037Z4MCM|Pyle Marine Sport...|
|B00P0BQKBC|Water & Wood Lock...|
|B00TP1C1UC|AmazonBasics 6-Ou...|
|B005AIF0X4|iHip MVF1030WO Ma...|
|B009JJKZSK|Click 'N Dig! Key...|
|B0097BEE9Q|Apple iPod nano 1...|
|B00KXDS4VQ|enKo products HDM...|
|B009W857SG|TETC Cable (3 Fee...|
|B007LG453Y|Tmvel TVM-UK2 Uni...|
|B00KFSO0SA|Monoprice 11149 3...|
|B00IH5U050|Digital Army Camo...|
|B00JDYMPYS|Sylvania SKCR2713...|
|B00YDMW0DQ|Sound Intone HD30...|
|B00AR0VCXW|Pro-Ject Speed Bo...|
|B006BE6WK6|12 PIECE Ear Hook...|
|B00TM1CT98|Bose Acoustimass ...|
|B00P8D911I|AuraBeam Samsung ...|
+----------+--------------------+
only showing top 20 rows



In [11]:
# Create  dataframe for apple
apple_df = cleaned_df.where(col('product_title').like("%Apple%"))
apple_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   38487968|R1EBPM82ENI67M|B000NU4OTA|      72265257|LIMTECH Wall char...|     Electronics|          1|            0|          0|   N|                Y|            One Star|Did not work at all.| 2015-08-31|
|         US|     657440|R1IUHD431Q5MSB|B00AY98USQ|     135712681|Lastest Apple TV ...|     Electronics|          5|    

In [12]:
# Create  dataframe for samsung
samsung_df = cleaned_df.where(col('product_title').like("%Samsung%"))
samsung_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20402068| RTDOEGCB6EQ5E|B0111Y2BW8|     761797447|White EZCast i5+ ...|     Electronics|          3|            0|          0|   N|                Y|It's just OK, but...|I've decided to k...| 2015-08-31|
|         US|   30909410| ROSNGEJRRBNYB|B004E2XB3I|     295457644|Mount-It! TV Wall...|     Electronics|          5|    

In [13]:
 # Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to active_user table in RDS

apple_df.write.jdbc(url=jdbc_url, table='gopro', mode=mode, properties=config)
samsung_df.write.jdbc(url=jdbc_url, table='canon', mode=mode, properties=config)
review_df.write.jdbc(url=jdbc_url, table='review', mode=mode, properties=config)
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)